<a href="https://colab.research.google.com/github/bonly/AI/blob/main/quant.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

<a href="https://colab.research.google.com/github/bonly/AI/blob/main/quant.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
%pip install akshare --upgrade

In [20]:
import akshare as ak;
from pprint import pprint;
help(ak.stock_zh_index_daily_tx)
# pprint(dir(ak.stock_zh_index_daily_tx))
# pprint(ak.__dict__)

Help on function stock_zh_index_daily_tx in module akshare.index.index_stock_zh:

stock_zh_index_daily_tx(symbol: str = 'sz980017') -> pandas.core.frame.DataFrame
    腾讯证券-日频-股票或者指数历史数据
    作为 ak.stock_zh_index_daily() 的补充, 因为在新浪中有部分指数数据缺失
    注意都是: 前复权, 不同网站复权方式不同, 不可混用数据
    https://gu.qq.com/sh000919/zs
    :param symbol: 带市场标识的股票或者指数代码
    :type symbol: str
    :return: 前复权的股票和指数数据
    :rtype: pandas.DataFrame



In [23]:

his = ak.stock_zh_a_hist(symbol="000001", period="daily", start_date="20170301", end_date='20240528', adjust="");
pprint(his.tail);


<bound method NDFrame.tail of               日期    股票代码     开盘     收盘     最高     最低      成交量           成交额  \
0     2017-03-01  000001   9.49   9.49   9.55   9.47   346994  3.301580e+08   
1     2017-03-02  000001   9.51   9.43   9.54   9.42   403629  3.823959e+08   
2     2017-03-03  000001   9.41   9.40   9.43   9.36   342655  3.219525e+08   
3     2017-03-06  000001   9.40   9.45   9.46   9.39   404511  3.812123e+08   
4     2017-03-07  000001   9.44   9.45   9.46   9.40   294673  2.777474e+08   
...          ...     ...    ...    ...    ...    ...      ...           ...   
1755  2024-05-22  000001  11.56  11.56  11.74  11.46  2115531  2.458449e+09   
1756  2024-05-23  000001  11.53  11.40  11.59  11.37  1841623  2.110799e+09   
1757  2024-05-24  000001  11.37  11.31  11.49  11.30  1398276  1.593330e+09   
1758  2024-05-27  000001  11.31  11.51  11.53  11.31  1454361  1.663272e+09   
1759  2024-05-28  000001  11.50  11.40  11.58  11.36  1204323  1.377107e+09   

        振幅   涨跌幅   涨跌